## Load files from Azure blob storage


### Step 1: Set the data location and type


In [0]:
storage_account_name = "mymlprojects"
storage_account_access_key = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-02-19T22:40:27Z&st=2024-02-01T14:40:27Z&spr=https&sig=PbOHAKAbxBdgR5Tw5g4pyWqBof3GolUQNoe%2BkAxDPPQ%3D"

In [0]:
storage_name = "wasbs://raw"

In [0]:
spark.conf.set(
  f"fs.azure.account.key{storage_account_name}.blob.core.windows.net",
  storage_account_access_key)

In [0]:
# Specify the path to the Azure Storage container or folder
container_path = f"wasbs://raw@{storage_account_name}.blob.core.windows.net/"


In [0]:
# Define the mount point and the storage account details
mount_point = "/mnt/ml_mount"
container_name = "raw"

# # Mount the Azure Blob Storage to DBFS using SAS token
# dbutils.fs.mount(
#   source = "wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/",
#   mount_point = mount_point,
#   extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net":
#     storage_account_access_key}
# )

In [0]:
# Files paths
dbutils.fs.ls(mount_point)